# Get Trader Joe's locations

In [1]:
import us
import requests
import pandas as pd
import geopandas as gpd

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
%load_ext lab_black

---

## Read data

In [4]:
headers = {
    "authority": "alphaapi.brandify.com",
    "accept": "application/json, text/plain, */*",
    "referer": "https://www.traderjoes.com/",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
}

#### Trader Joe's has nice json data without limits. Thanks, Trader Joe's. 

In [5]:
json_data = {
    "request": {
        "appkey": "8BC3433A-60FC-11E3-991D-B2EE0C70A832",
        "formdata": {
            "geoip": False,
            "dataview": "store_default",
            "limit": 1000,
            "geolocs": {
                "geoloc": [
                    {
                        "addressline": "90066",
                        "country": "US",
                        "latitude": "",
                        "longitude": "",
                    },
                ],
            },
            "searchradius": "5000",
            "where": {
                "warehouse": {
                    "distinctfrom": "1",
                },
            },
            "false": "0",
        },
    },
}

response = requests.post(
    "https://alphaapi.brandify.com/rest/locatorsearch", headers=headers, json=json_data
)

#### Dataframe

In [6]:
src = pd.DataFrame(response.json()["response"]["collection"])

#### How many locations did we capture? 

In [7]:
len(src)

551

#### Tidy up

In [8]:
df = src[
    [
        "uid",
        "name",
        "address1",
        "city",
        "state",
        "postalcode",
        "phone",
        "latitude",
        "longitude",
        "beer",
        "liquor",
        "wineshop",
        "regions",
    ]
].copy()

In [9]:
df.rename(columns={"address1": "address", "postalcode": "zip"}, inplace=True)

In [10]:
df["uid"] = df["uid"].abs().astype(str)[0]

In [11]:
df["store_number"] = (
    df["name"]
    .str[-4:]
    .str.strip()
    .str.replace(")", "", regex=False)
    .str.replace("(", "", regex=False)
    .astype(str)
)

In [12]:
df["name"] = df["name"].str.split("(", expand=True)[0]

#### The result: 

In [13]:
df.head()

,uid,name,address,city,state,zip,phone,latitude,longitude,beer,liquor,wineshop,regions,store_number
0,1429860783,Trader Joe's West Los Angeles - Sepulveda,3456 S Sepulveda Blvd,Los Angeles,CA,90034,310-836-2458,34.020480,-118.422106,Yes,Yes,None,WestSide/SouthBay,119
1,1429860783,Trader Joe's Marina Del Rey,4675 Admiralty Way,Marina Del Rey,CA,90292,310-301-4461,33.979559,-118.441227,Yes,Yes,None,WestSide/SouthBay,205
2,1429860783,Trader Joe's West Los Angeles,10850 National Blvd,Los Angeles,CA,90064,310-470-1917,34.030059,-118.421482,Yes,Yes,None,WestSide/SouthBay,7
3,1429860783,Trader Joe's Santa Monica,3212 Pico Blvd,Santa Monica,CA,90405,310-581-0253,34.026355,-118.455765,Yes,Yes,None,WestSide/SouthBay,6
4,1429860783,Trader Joe's Culver City,6150 Slauson Ave,Culver City,CA,90230,310-313-0441,33.988010,-118.392850,Yes,Yes,None,WestSide/SouthBay,251


---

## Geography

#### Make it a geodataframe

In [14]:
df_geo = df.copy()

In [15]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [16]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

#### Counties

In [17]:
counties = gpd.read_file("data/raw/usa_counties_esri_simple.json")

In [18]:
counties_gdf = counties[
    [
        "fid",
        "name",
        "state_name",
        "state_fips",
        "cnty_fips",
        "fips",
        "population",
        "geometry",
    ]
].copy()

In [19]:
tj_locations_counties = locations_gdf.sjoin(counties, how="left", predicate="within")

In [20]:
counties_grouped = (
    (
        tj_locations_counties.groupby(["name_right", "cnty_fips", "city", "state"])[
            "uid"
        ]
        .count()
        .reset_index()
        .sort_values("uid", ascending=False)
    )
    .rename(columns={"uid": "count"})
    .reset_index(drop=True)
)

In [21]:
counties_grouped.head()

,name_right,cnty_fips,city,state,count
0,Los Angeles County,037,Los Angeles,CA,11
1,New York County,061,New York,NY,9
2,San Diego County,073,San Diego,CA,8
3,San Francisco County,075,San Francisco,CA,6
4,Cook County,031,Chicago,IL,6


---

## Exports

#### CSV

In [22]:
df.to_csv("data/processed/trader_joes_locations.csv", index=False)

#### JSON

In [23]:
df.to_json("data/processed/trader_joes_locations.json", indent=4, orient="records")

#### GeoJSON

In [24]:
locations_gdf.to_file("data/processed/trader_joes_locations.geojson", driver="GeoJSON")